In [26]:
from minpack_numba import lmdif1, sig_minpack_func2
from NumbaMinpack import lmdif, minpack_sig

from numba import cfunc
import numba as nb
import numpy as np
import ctypes as ct

import cffi

ffi = cffi.FFI()


@cfunc(sig_minpack_func2)
def myfunc2(m, n, x, fvec, iflag, udata):
    x = nb.carray(x, n, np.float64)
    fvec = nb.carray(fvec, m, np.float64)
    fvec[0] = x[0] ** 2 - 30.0 * x[1]
    fvec[1] = x[1] ** 2 - 8.0 * x[0]
    fvec[2] = x[2] ** 2 - 5.0 * x[1]
    # fvec[:] = x ** 2
    # fvec[0] -= 30.0 * x[1]
    # fvec[1] -= 8.0 * x[0]
    # fvec[2] -= 5.0 * x[1]
    # fvec[(0, 2)] -= udata[0]

    # # print(udata[0])
    # fvec[0] = x[0] ** 2 - 30.0 * x[1] - nb.carray(udata, 2, np.float64).sum()
    # fvec[1] = x[1] ** 2 - 8.0 * x[0] - udata[0]
    # fvec[2] = x[2] ** 2 - 5.0 * x[1] + udata[1]
    # # fvec[m] *= np.pi
    # # print("hello", m)
    # for i in range(m-1):
    #     fvec[i] *= np.pi
    #     fvec[i] += i * 1000

    # # print("hello")
    # iflag[0] = -1


@cfunc(minpack_sig)
def myfunc(x, fvec, udata):
    fvec[0] = x[0] ** 2 - 30.0 * x[1]
    fvec[1] = x[1] ** 2 - 8.0 * x[0]
    fvec[2] = x[2] ** 2 - 5.0 * x[1]

    # fvec[0] = x[0] ** 2 - 30.0 * x[1] - udata[0] - udata[1]
    # fvec[1] = x[1] ** 2 - 8.0 * x[0] - udata[0]
    # fvec[2] = x[2] ** 2 - 5.0 * x[1] + udata[1]

    # fvec[0] *= np.pi
    # fvec[1] *= np.pi
    # # fvec[2] *= np.pi

    # fvec[0] += 0
    # fvec[1] += 1000
    # fvec[2] += 2000


x_init = np.array([10.0, 10.0, 10.0])  # initial conditions
neqs = 3  # number of equations
args = np.array([30.0, 8.0])

lmdif1(myfunc2.address, a := x_init.copy(), neqs, 3, udata=args), lmdif(
    myfunc.address, x_init, neqs, args=args
), a

((array([19.30978769, 12.42893002,  7.88318781]),
  array([ 5.68434189e-14,  0.00000000e+00, -2.13162821e-14]),
  True,
  2),
 (array([19.30978769, 12.42893002,  7.88318781]),
  array([ 5.68434189e-14,  0.00000000e+00, -7.10542736e-15]),
  True,
  2),
 array([19.30978769, 12.42893002,  7.88318781]))

In [3]:
from minpack_numba.minpack_numba import _lmdif1

_lmdif1.argtypes

[ctypes.c_int,
 ctypes.c_int,
 ctypes.c_int,
 ctypes.c_void_p,
 ctypes.c_void_p,
 ctypes.c_double,
 ctypes.c_int,
 ctypes.c_int,
 ctypes.c_int,
 ctypes.c_int,
 ctypes.c_void_p]

In [25]:
a = [np.array([1, 2]), np.array([3, 4])]
sum(a)

array([4, 6])

In [2]:
%timeit lmdif1(myfunc2.address, x_init.copy(), neqs, 3, udata=args)
%timeit lmdif(myfunc.address, x_init, neqs, args=args)

3.99 µs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
4.2 µs ± 20.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [3]:
import ctypes
import numba as nb
import numpy as np

arr = np.arange(5).astype(np.double)  # create arbitrary numpy array
print(arr)


@nb.extending.intrinsic
def address_as_void_pointer(typingctx, src):
    """returns a void pointer from a given memory address"""
    from numba.core import types, cgutils

    sig = types.voidptr(src)

    def codegen(cgctx, builder, sig, args):
        return builder.inttoptr(args[0], cgutils.voidptr_t)

    return sig, codegen


addr = arr.ctypes.data


@nb.njit
def modify_data():
    """a function taking the memory address of an array to modify it"""
    data = nb.carray(address_as_void_pointer(addr), arr.shape, dtype=arr.dtype)
    data += 2


modify_data()
print(arr)

[0. 1. 2. 3. 4.]
[2. 3. 4. 5. 6.]


In [4]:
address_as_void_pointer()

NotImplementedError: <intrinsic address_as_void_pointer> is not usable in pure-python

In [4]:
ffi.from_buffer(args.astype(np.float64))

<cdata 'char[]' buffer len 16 from 'numpy.ndarray' object>

In [5]:
ct.POINTER((args,))

TypeError: unhashable type: 'numpy.ndarray'

In [2]:
import ctypes
import numpy

c_float_p = ctypes.POINTER(ctypes.c_float)
data = numpy.array([[0.1, 0.1], [0.2, 0.2], [0.3, 0.3]])
data = data.astype(numpy.float32)
data_p = data.ctypes.data_as(c_float_p)

In [2]:

%timeit lmdif1(myfunc2.address, x_init.copy(), neqs,3, udata=args)
%timeit lmdif(myfunc.address, x_init, neqs, args=args)

4.03 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
4.25 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
from time import sleep
%timeit sleep(0.1); sleep(0.2)
# ... sleep(0.3)

In [ ]:
import numba as nb
import numpy as np

z = np.zeros((10,), dtype=np.int32)

print(z.size)

@nb.njit
def wot(a):
    return np.int32(a.size)

print(wot(z))
%timeit wot(z)

In [1]:
import ctypes as ct
import numpy as np
from numba import types, njit, cfunc, carray
from minpack_numba import sig_minpack_func2

cminpack = ct.CDLL(ct.util.find_library("cminpack"))

In [2]:
# int __cminpack_func__(lmdif1)(__cminpack_decl_fcn_mn__ void *p,
# int m,
# int n,
# real *x,
# 	real *fvec,
# real tol,
# int *iwa,
# 	real *wa,
# int lwa)

lmdif69 = cminpack.lmdif1
lmdif69.argtypes = [
    ct.c_void_p,  # *p
    ct.c_int,  # m
    ct.c_int,  # n
    ct.c_double,  # *x
    ct.c_double,  # *fvec
    ct.c_double,  # tol
    ct.c_int,  # *iwa
    ct.c_double,  # *wa
    ct.c_int,  # lwa
]

# lmdif69.argtypes = [
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p,
#     ct.c_void_p
# ]
lmdif69.restype = ct.c_int

In [3]:
# @njit
def wot(
    func,
    x0,
    m,
    n=None,
    tol=1.49012e-8,
    udata=None,
):
    tol = np.float64(tol)
    # tol = np.array(tol, dtype=np.float64)

    # m = np.array(m, dtype=np.int32)
    # n = np.array(x0.size, dtype=np.int32)

    m = np.int32(m)
    if n is None:
        n = np.int32(x0.size)
    else:
        n = np.int32(n)

    fvec = np.zeros(m, dtype=np.float64)

    info = np.zeros(1, dtype=np.int32)
    iwa = np.zeros(n, dtype=np.int32)
    # iwa[0] = -1
    # lwa = np.int32((m * n + 5 * n + m) + 2)
    lwa = np.array((m * n + 5 * n + m) + 2, dtype=np.int32)
    wa = np.zeros(lwa.item(), dtype=np.float64)

    if udata is None:
        udata = np.zeros(1, dtype=np.float64)
    else:
        udata = np.asarray(udata)

    # func(m, n, x0.ctypes.data, ct.POINTER(fvec.ctypes.data), 0)

    lmdif69(
        func,
        m,
        n,
        x0.ctypes.data,  #
        fvec.ctypes.data,  #
        tol,
        # info.ctypes.data,#
        iwa.ctypes.data,  #
        wa.ctypes.data,  #
        lwa,
        # udata.ctypes.data,#
    )

    # lmdif69(
    #     func,
    #     m.ctypes.data,
    #     n.ctypes.data,
    #     x0.ctypes.data,#
    #     fvec.ctypes.data,#
    #     tol.ctypes.data,
    #     # info.ctypes.data,#
    #     iwa.ctypes.data,#
    #     wa.ctypes.data,#
    #     lwa.ctypes.data,
    #     # udata.ctypes.data,#
    # )
    return x0, fvec, (1 <= info <= 4)[0], info[0]

In [9]:
sig = types.void(
    types.int32,
    types.int32,
    types.CPointer(types.double),
    types.CPointer(types.double),
    types.int32,
    # types.CPointer(types.double),
)


@cfunc(sig)
def myfunc69(m, n, x, fvec, iflag):  # , udata):
    # print("woo")
    x[0] = 1
    fvec[0] = 1
    print("woop")
    iflag = -1
    # return
    # fvec[0] = 1
    # _fvec = carray(fvec, m)
    # print("hello")
    # print(fvec)
    # fvec[0] = x[0]**2 - 30.0*x[1]
    # fvec[1] = x[1]**2 - 8.0*x[0]
    # fvec[2] = x[2]**2  - 5.0*x[1]
    # print(fvec)
    # return iflag.ctypes.data


x_init = np.array([10.0, 10.0, 10.0])  # initial conditions
neqs = 3  # number of equations
args = np.array([30.0, 8.0])

In [10]:
wot(myfunc69.address, x_init.copy(), neqs, 3, udata=args)

(array([10., 10., 10.]), array([0., 0., 0.]), False, 0)

In [ ]:
# subroutine minpack_fcn_lmder(m, n, x, fvec, fjac, ldfjac, iflag, udata) bind(c)
#     import :: c_int, c_double, c_ptr
#     implicit none
#     integer(c_int), value :: m
#     integer(c_int), value :: n
#     integer(c_int), value :: ldfjac
#     integer(c_int), intent(inout) :: iflag
#     real(c_double), intent(in) :: x(n)
#     real(c_double), intent(inout) :: fvec(m)
#     real(c_double), intent(inout) :: fjac(ldfjac, n)
#     type(c_ptr), value :: udata
# end subroutine minpack_fcn_lmder

# subroutine minpack_lmder1(fcn, m, n, x, Fvec, Fjac, Ldfjac, Tol, Info, Ipvt, Wa, Lwa, &
#         & udata) &
#         & bind(c)
#     procedure(minpack_fcn_lmder) :: fcn
#     integer(c_int), value :: m
#     integer(c_int), value :: n
#     integer(c_int), value :: ldfjac
#     integer(c_int), intent(out) :: info
#     integer(c_int), value :: lwa
#     integer(c_int), intent(out) :: ipvt(n)
#     real(c_double), value :: tol
#     real(c_double), intent(inout) :: x(n)
#     real(c_double), intent(out) :: fvec(m)
#     real(c_double), intent(out) :: fjac(ldfjac, n)
#     real(c_double), intent(inout) :: wa(lwa)
#     type(c_ptr), value :: udata

#     call lmder1(wrap_fcn, m, n, x, Fvec, Fjac, Ldfjac, Tol, Info, Ipvt, Wa, Lwa)

# contains
#     subroutine wrap_fcn(m, n, x, fvec, fjac, ldfjac, iflag)
#         integer, intent(in) :: m
#         integer, intent(in) :: n
#         integer, intent(in) :: ldfjac
#         integer, intent(inout) :: iflag
#         real(wp), intent(in) :: x(n)
#         real(wp), intent(inout) :: fvec(m)
#         real(wp), intent(inout) :: fjac(ldfjac, n)

#         call fcn(m, n, x, fvec, fjac, ldfjac, iflag, udata)
#     end subroutine wrap_fcn
# end subroutine minpack_lmder1

In [1]:
import ctypes as ct
from numba import njit, types, cfunc
import numpy as np

minpack = ct.CDLL(ct.util.find_library("minpack"))

# subroutine minpack_lmder1(fcn, m, n, x, Fvec, Fjac, Ldfjac, Tol, Info, Ipvt, Wa, Lwa, &
#         & udata) &
#         & bind(c)
#     procedure(minpack_fcn_lmder) :: fcn
#     integer(c_int), value :: m
#     integer(c_int), value :: n
#     integer(c_int), value :: ldfjac
#     integer(c_int), intent(out) :: info
#     integer(c_int), value :: lwa
#     integer(c_int), intent(out) :: ipvt(n)
#     real(c_double), value :: tol
#     real(c_double), intent(inout) :: x(n)
#     real(c_double), intent(out) :: fvec(m)
#     real(c_double), intent(out) :: fjac(ldfjac, n)
#     real(c_double), intent(inout) :: wa(lwa)
#     type(c_ptr), value :: udata

#     call lmder1(wrap_fcn, m, n, x, Fvec, Fjac, Ldfjac, Tol, Info, Ipvt, Wa, Lwa)

# contains
#     subroutine wrap_fcn(m, n, x, fvec, fjac, ldfjac, iflag)
#         integer, intent(in) :: m
#         integer, intent(in) :: n
#         integer, intent(in) :: ldfjac
#         integer, intent(inout) :: iflag
#         real(wp), intent(in) :: x(n)
#         real(wp), intent(inout) :: fvec(m)
#         real(wp), intent(inout) :: fjac(ldfjac, n)

#         call fcn(m, n, x, fvec, fjac, ldfjac, iflag, udata)
#     end subroutine wrap_fcn
# end subroutine minpack_lmder1


sig_minpack_fcn_lmder = types.void(
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
)

_lmder1 = minpack.minpack_lmder1
_lmder1.argtypes = [
    ct.c_void_p,
    ct.c_int,
    ct.c_int,
    ct.c_int,
    ct.c_void_p,
    ct.c_int,
    ct.c_void_p,
    ct.c_double,
    ct.c_void_p,
    ct.c_void_p,
    ct.c_void_p,
    ct.c_void_p,
    ct.c_void_p,
]
_lmder1.restype = None

# subroutine minpack_fcn_lmder(m, n, x, fvec, fjac, ldfjac, iflag, udata) bind(c)
#     import :: c_int, c_double, c_ptr
#     implicit none
#     integer(c_int), value :: m
#     integer(c_int), value :: n
#     integer(c_int), value :: ldfjac
#     integer(c_int), intent(inout) :: iflag
#     real(c_double), intent(in) :: x(n)
#     real(c_double), intent(inout) :: fvec(m)
#     real(c_double), intent(inout) :: fjac(ldfjac, n)
#     type(c_ptr), value :: udata
# end subroutine minpack_fcn_lmder

sig_minpack_fcn_lmder = types.void(
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
    types.CPointer(types.float64),
)

#     procedure(minpack_fcn_lmder) :: fcn
#     integer(c_int), value :: m
#     integer(c_int), value :: n
#     integer(c_int), value :: ldfjac
#     integer(c_int), intent(out) :: info
#     integer(c_int), value :: lwa
#     integer(c_int), intent(out) :: ipvt(n)
#     real(c_double), value :: tol
#     real(c_double), intent(inout) :: x(n)
#     real(c_double), intent(out) :: fvec(m)
#     real(c_double), intent(out) :: fjac(ldfjac, n)
#     real(c_double), intent(inout) :: wa(lwa)
#     type(c_ptr), value :: udata


@njit
def lmder1(func, x0, m, ldfjac, tol=1.49012e-8, udata=None):
    x = np.asarray(x0.copy(), dtype=np.float64)

    m = np.int32(m)
    n = np.int32(x.size)
    ldfjac = np.int32(ldfjac)
    info = np.zeros(1, dtype=np.int32)
    lwa = np.int32(5 * n + m)
    ipvt = np.zeros(n, dtype=np.int32)
    tol = np.float64(tol)
    fvec = np.zeros(m, dtype=np.float64)
    # fjac = np.zeros((ldfjac, n), dtype=np.float64)
    fjac = np.zeros((n, ldfjac), dtype=np.float64)
    wa = np.zeros(lwa, dtype=np.float64)
    udata = np.zeros(1, dtype=np.float64)

    _lmder1(
        func,
        m,
        n,
        ldfjac,
        info.ctypes.data,
        lwa,
        ipvt.ctypes.data,
        tol,
        x0.ctypes.data,
        fvec.ctypes.data,
        fjac.ctypes.data,
        wa.ctypes.data,
        udata.ctypes.data,
    )

    return x0, fvec, (1 <= info <= 4)[0], info[0]


@cfunc(sig_minpack_fcn_lmder)
def myfunc69(m, n, x, fvec, fjac, ldfjac, iflag, udata):
    # fvec[0] = x[0] ** 2 - 30.0 * x[1]
    # fvec[1] = x[1] ** 2 - 8.0 * x[0]
    # fvec[2] = x[2] ** 2 - 5.0 * x[1]
    # print("woop")
    # udata[0] += 1

    # if udata[0] > 10:
    #     iflag[0] = -1
    print("hello")
    iflag[0] = -1


x_init = np.array([10.0, 10.0])  # initial conditions
neqs = 3  # number of equations
args = np.array([30.0, 8.0])


lmder1(myfunc69.address, x_init, 3, 3, udata=args)

: 

: 

In [4]:
import numpy as np
import numba as nb
import ctypes as ct

import xraylib_np as xrl_np

rng = np.random.default_rng(0)

xrl_numba = ct.CDLL(ct.util.find_library("libxrl"))

ElementDensity = xrl_numba.ElementDensity
ElementDensity.argtypes = [ct.c_int]
ElementDensity.restype = ct.c_double

x = rng.integers(1, 90, size=10000)

@nb.njit(parallel=True)
def ElementDensity2(Z):
    output = np.zeros_like(Z, dtype=np.float64)
    for i in range(Z.shape[0]):
        output[i] = ElementDensity(Z[i])
    return output

ElementDensity2(x) - xrl_np.ElementDensity(x)

%timeit ElementDensity2(x)
%timeit xrl_np.ElementDensity(x)

/Users/chris/mambaforge/envs/mbipy310/lib/python3.10/site-packages/numba/core/typed_passes.py:334: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../var/folders/v7/vq2l7f812yd450mn3wwmrhtc0000gn/T/ipykernel_30847/2565436052.py", line 17:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


14.1 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
159 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [46]:
import ctypes as ct
import numba as nb
import xraylib as xrl
import numpy as np

AtomicLevelWidth = ct.CDLL(ct.util.find_library("libxrl")).AtomicLevelWidth
AtomicLevelWidth.argtypes = [ct.c_int, ct.c_int]#, ct.c_void_p]
AtomicLevelWidth.restype = ct.c_double

AtomicLevelWidth2 = ct.PyDLL(ct.util.find_library("libxrl")).AtomicLevelWidth
AtomicLevelWidth2.argtypes = [ct.c_int, ct.c_int, ct.c_void_p]
AtomicLevelWidth2.restype = ct.c_double



# double Refractive_Index_Re(const char compound[], double E, double density, xrl_error **error)
Refractive_Index_Re = ct.CDLL(ct.util.find_library("libxrl")).Refractive_Index_Re
Refractive_Index_Re.argtypes = [ct.c_char_p, ct.c_double, ct.c_double]


@nb.njit("float64(int32, int64)")#(nogil=True)
# @nb.njit("float64(int32, int32)")#(nogil=True)
def AtomicLevelWidth_numba(Z, shell):
    # error = np.array([0], dtype=np.int32)
    result = AtomicLevelWidth(Z, shell)#, 0)
    if result:
        return result
    raise ValueError("Error in AtomicLevelWidth")
    
    # if error[0]:
    #     raise ValueError("Error in AtomicLevelWidth")
    # return result

@nb.njit
def AtomicLevelWidth_numba2(Z, shell):
    error = np.array([0], dtype=np.int32)
    result = AtomicLevelWidth2(Z, shell, error.ctypes.data)
    if error[0]:
        raise ValueError("Error in AtomicLevelWidth")
    return result


AtomicLevelWidth_numba(26, 1)#, xrl.AtomicLevelWidth(26, -1)
AtomicLevelWidth_numba2(26, 1)#, xrl.AtomicLevelWidth(26, -1)
%timeit AtomicLevelWidth_numba(26, 1)
%timeit AtomicLevelWidth_numba2(26, 1)
%timeit xrl.AtomicLevelWidth(26, 1)

83.7 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


KeyboardInterrupt: 

In [30]:
ct.CDLL(ct.util.find_library("libxrl")).KA_LINE

AttributeError: dlsym(0x9fc4d980, KA_LINE): symbol not found

In [107]:
import ctypes as ct

ct.CDLL(ct.util.find_library("libxrl")), ct.PyDLL(
    ct.util.find_library("libxrl")
)

(<CDLL '/usr/local/lib/libxrl.dylib', handle 9dc0f220 at 0x2a4c66ef0>,
 <PyDLL '/usr/local/lib/libxrl.dylib', handle 9dc0f220 at 0x2a4c66680>)

In [108]:
ct.PyDLL(ct.util.find_library("libxrl")).AtomicLevelWidth

In [5]:
import ctypes as ct

Fi = ct.CDLL(ct.util.find_library("libxrl")).Fi
import xraylib as xrl
import xraylib_np as xrl_np
import numpy as np
import numba as nb

rng = np.random.default_rng()
Fi.argtypes = [ct.c_int, ct.c_double]
Fi.restype = ct.c_double


# float64[:, :](int64[:], float64[:], optional(float64[:, :]))",
@nb.njit(parallel=True, nogil=True)
def Fi69(x, y, out=None):
    assert x.ndim == 1
    assert y.ndim == 1
    # assert x.dtype == np.int64
    # assert y.dtype == "np.float64"

    null = np.array([1], dtype=np.int32)

    x_size = x.size
    y_size = y.size
    if out is None:
        output = np.zeros((x.size, y.size), dtype=y.dtype)
    else:
        assert out.shape == (x.size, y.size)
        output = out

    # for ij in nb.prange(x_size*y_size):
    #     j = ij % y_size
    #     i = ij // y_size

    for i in range(x.size):
        for j in range(y.size):
            output[i, j] = Fi(x[i], y[j])
    return output
    # for i in range(x.size):
    #     for j in range(y.size):
    #         output[i, j] = Fi(x[i], y[j])
    # return output

    # return Fi(x.ctypes.data, y.ctypes.data)
    # return Fi(x, y)


x = np.arange(1, 100, dtype=np.int64)
y = rng.random(101).astype(np.float64)

# Fi2(1, 2.), xrl.Fi(1, 2.)
# %timeit Fi2(1, 2.)
# %timeit xrl.Fi(1, 2.)
# nb.numba.NULL
Fi69(x, y)

array([[ 1.52934002e-03,  2.77989059e-03,  9.78829835e-03, ...,
         6.31206031e-03,  6.39407431e-03,  1.90891785e-02],
       [ 2.46380731e-02,  4.03640805e-02,  1.01206382e-01, ...,
         7.55110875e-02,  7.61857992e-02,  1.47577219e-01],
       [ 1.00658057e-01,  1.53643271e-01,  3.03273460e-01, ...,
         2.52618295e-01,  2.54232091e-01,  3.18430149e-01],
       ...,
       [-8.03213797e+01, -7.45811509e+01, -6.29330144e+01, ...,
        -6.61891943e+01, -6.60593477e+01, -5.43115985e+01],
       [-8.83805839e+01, -7.45453216e+01, -6.32232980e+01, ...,
        -6.69343498e+01, -6.68026736e+01, -5.38362504e+01],
       [-9.55728177e+01, -7.34366952e+01, -6.24246593e+01, ...,
        -6.67506443e+01, -6.66264278e+01, -5.39725643e+01]])

In [6]:
assert np.allclose(Fi69(x, y), xrl_np.Fi(x.astype(np.int64), y))
assert np.allclose(Fi69(x, y, np.zeros((x.size, y.size))), xrl_np.Fi(x.astype(np.int64), y))

%timeit Fi69(x, y)
%timeit Fi69(x, y, np.zeros((x.size, y.size)))
%timeit xrl_np.Fi(x, y)


371 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
275 µs ± 5.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
171 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [98]:
import numba as nb
import numpy as np

@nb.njit
def wot(a, b):
    return np.zeros((a, b), dtype=np.float64)

wot(100, 200)
%timeit wot(100, 200)
%timeit np.zeros((100, 200), dtype=np.float64)

1.77 µs ± 7.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.78 µs ± 1.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [86]:
%timeit np.zeros((100, 100), dtype=np.float64)

1.21 µs ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
import ctypes as ct
import numpy as np
import numba as nb
from numba import njit, types, cfunc
from minpack_numba.minpack_numba import (
    _address_as_void_pointer,
    lmdif1,
    sig_minpack_func2,
)

path = "/Users/chris/Documents/PhD/minpack_numba/minpack/libminpack_numba.so"

minpack_numba = ct.CDLL(path)
_lmder1 = minpack_numba.lmder1_numba


_lmder1.restype = None

sig = types.void(
    types.int32,
    types.int32,
    types.CPointer(types.double),
    types.CPointer(types.double),
    types.CPointer(types.double),
    types.int32,
    types.CPointer(types.int32),
    types.CPointer(types.double),
)


@cfunc(sig)
def func(m, n, x, fvec, fjac, ldfjac, iflag, udata):
    fjac = nb.farray(fjac, (ldfjac, n), dtype=np.float64)
    if iflag[0] == 1:
        fvec[0] = x[0] ** 2 - 30.0 * x[1] - udata[0]
        fvec[1] = x[1] ** 2 - 8.0 * x[0]
        fvec[2] = x[2] ** 2 - 5.0 * x[1]
    if iflag[0] == 2:
        fjac[0, 0] = 2.0 * x[0]
        fjac[0, 1] = -30.0
        fjac[1, 0] = -8.0
        fjac[1, 1] = 2.0 * x[1]
        fjac[2, 1] = -5.0
        fjac[2, 2] = 2.0 * x[2]
        # TODO fjac
        # print("bonjour")

    nb.carray(x, n, dtype=np.float64)
    nb.carray(udata, ldfjac, dtype=np.float64)


@cfunc(sig_minpack_func2)
def func2(m, n, x, fvec, iflag, udata):
    fvec[0] = x[0] ** 2 - 30.0 * x[1] - udata[0]
    fvec[1] = x[1] ** 2 - 8.0 * x[0]
    fvec[2] = x[2] ** 2 - 5.0 * x[1]


_lmder1.argtypes = [
    ct.c_void_p,  # fcn
    ct.c_int,  # m
    ct.c_int,  # n
    ct.c_void_p,  # x
    ct.c_void_p,  # fvec
    ct.c_void_p,  # fjac
    ct.c_int,  # ldfjac
    ct.c_double,  # tol
    ct.c_int,  # info
    ct.c_int,  # ipvt
    ct.c_void_p,  # wa
    ct.c_int,  # lwa
    ct.c_void_p,  # udata
]


@njit
def wot(fcn, udata):
    n = np.int32(3)
    m = np.int32(3)
    ldfjac = np.int32(3)
    tol = np.float64(1.49012e-8)
    x = 100.0 * np.ones(n, dtype=np.float64)  # * 10.0
    fvec = np.zeros(m, dtype=np.float64)
    fjac = np.zeros((ldfjac, n), dtype=np.float64)
    # lwa = np.int32(m * (n + 1) + 2 * n)
    ipvt = np.zeros(n, dtype=np.int32)
    lwa = np.int32(5 * n + m)
    wa = np.zeros(lwa, dtype=np.float64)
    info = np.zeros(1, dtype=np.int32)
    # udata = np.zeros(100, dtype=np.float64)
    _lmder1(
        fcn,
        m,
        n,
        _address_as_void_pointer(x.ctypes.data),
        _address_as_void_pointer(fvec.ctypes.data),
        _address_as_void_pointer(fjac.ctypes.data),
        ldfjac,
        tol,
        info.ctypes.data,
        ipvt.ctypes.data,
        wa.ctypes.data,
        lwa,
        _address_as_void_pointer(udata.ctypes.data),
    )
    # return info
    return x, fvec, fjac, info, ipvt


udata = np.array([30.0, 8.0])

(a := wot(func.address, udata)), lmdif1(
    func2.address, 10.0 * np.ones(3, dtype=np.float64), 3, 3, udata=udata
)

((array([20.30701395, 12.74582724,  7.98305307]),
  array([-1.44607577e-06,  3.56487760e-07,  5.20454009e-08]),
  array([[ 42.74180401,  -0.22202828,  -0.37354778],
         [ 30.81154718, -25.0094584 ,  -0.67781267],
         [-38.4948964 ,   9.37229707, -11.99759208]]),
  array([2], dtype=int32),
  array([3, 2, 1], dtype=int32)),
 (array([20.30701399, 12.74582723,  7.98305306]),
  array([0., 0., 0.]),
  True,
  2))

In [ ]:
wot(func.address, udata)

: 

: 

In [ ]:
%timeit wot(func.address, udata)
%timeit lmdif1(func2.address, np.ones(3, dtype=np.float64), 3, 3, udata=udata)

: 

: 

In [27]:
import ctypes as ct
import numpy as np
import numba as nb
from numba import njit, types, cfunc
from minpack_numba.minpack_numba import _address_as_void_pointer

minpack_path = ct.util.find_library("minpack")
print(minpack_path)
minpack = ct.CDLL(minpack_path)

sig = types.void(
    types.int32,
    types.int32,
    types.CPointer(types.double),
    types.CPointer(types.double),
    types.CPointer(types.double),
    types.CPointer(types.int32),
    types.CPointer(types.int32),
    types.CPointer(types.double),
)


@cfunc(sig)
def func(m, n, x, fvec, fjac, ldfjac, iflag, udata):
    print("hello")
    # iflag[0] = -1


_lmder1 = minpack.minpack_lmder1
_lmder1.argtypes = [
    ct.c_int,  # fcn
    ct.c_int,  # m
    ct.c_int,  # n
    ct.c_int,  # x
    ct.c_int,  # fvec
    ct.c_int,  # fjac
    ct.c_int,  # ldfjac
    ct.c_double,  # tol
    ct.c_int,  # info
    ct.c_int,  # wa
    ct.c_int,  # lwa
    ct.c_int,  # udata
]
_lmder1.restype = None


@njit
def lmder1(fcn, x0, m, n=None, ldfjac=None, tol=1.49012e-8, udata=None):
    tol = np.float64(tol)

    x = x0.copy()

    m = np.int32(m)
    if n is None:
        n = np.int32(x.size)
    else:
        n = np.int32(n)

    if ldfjac is None:
        ldfjac = np.int32(m)
    else:
        ldfjac = np.int32(ldfjac)

    fvec = np.zeros(m, dtype=np.float64)
    fjac = np.zeros((ldfjac, n), dtype=np.float64)

    info = np.zeros(1, dtype=np.int32)
    iwa = np.zeros(n, dtype=np.int32)
    # (m * n + 5 * n + m)+2
    lwa = np.int32((m * n + 5 * n + m))  # m*n+5*n+m
    ipvt = np.zeros(n, dtype=np.int32)

    wa = np.zeros(lwa, dtype=np.float64)

    if udata is None:
        udata = np.zeros(1, dtype=np.float64)
    # else:
    #     udata = np.asarray(udata)

    _lmder1(
        fcn,
        m,
        n,
        x.ctypes.data,
        fvec.ctypes.data,
        fjac.ctypes.data,
        # _address_as_void_pointer(x.ctypes.data),
        # _address_as_void_pointer(fvec.ctypes.data),
        # _address_as_void_pointer(fjac.ctypes.data),
        ldfjac,
        tol,
        info.ctypes.data,
        ipvt.ctypes.data,
        wa.ctypes.data,
        lwa,
        udata.ctypes.data
        # _address_as_void_pointer(udata.ctypes.data)
    )
    return x0, fvec, fjac, (1 <= info <= 4)[0], info[0]


lmder1(func.address, np.ones(3) * 10, 3, 3, 3)

/usr/local/lib/libminpack.dylib


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function ExternalFunctionPointer((int32, int32, int32, int32, int32, int32, int32, float64, int32, int32, int32, int32) -> none) found for signature:
 
 >>> ExternalFunctionPointer(int64, int32, int32, uint64, uint64, uint64, int32, float64, uint64, uint64, uint64, int32, uint64)
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Type Restricted Function in function 'unknown': File: unknown: Line unknown.
    With argument(s): '(int64, int32, int32, uint64, uint64, uint64, int32, float64, uint64, uint64, uint64, int32, uint64)':[0m
[1m   No match for registered cases:
    * (int32, int32, int32, int32, int32, int32, int32, float64, int32, int32, int32, int32) -> none[0m
[0m
[0m[1mDuring: resolving callee type: ExternalFunctionPointer((int32, int32, int32, int32, int32, int32, int32, float64, int32, int32, int32, int32) -> none)[0m
[0m[1mDuri